In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pynwb
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb.epoch import TimeIntervals
from pynwb.file import Subject
from pynwb.behavior import SpatialSeries, Position
from datetime import datetime
from dateutil import tz